In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 6
batch_size = 512
device = 'cpu'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.629314422607422 | KNN Loss: 5.94550085067749 | CLS Loss: 1.6838135719299316
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 6.246893882751465 | KNN Loss: 5.410148620605469 | CLS Loss: 0.8367450833320618
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 5.863901138305664 | KNN Loss: 5.224198341369629 | CLS Loss: 0.6397028565406799
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 5.8153977394104 | KNN Loss: 5.172204494476318 | CLS Loss: 0.6431933641433716
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 5.770508289337158 | KNN Loss: 5.207658290863037 | CLS Loss: 0.5628498792648315
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 5.6319355964660645 | KNN Loss: 5.1199951171875 | CLS Loss: 0.5119404792785645
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 5.606156826019287 | KNN Loss: 5.101445198059082 | CLS Loss: 0.5047116279602051
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 5.439274787902832 | KNN Loss: 5.02860164642334 | CLS Loss: 0

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 5.045559406280518 | KNN Loss: 4.834207057952881 | CLS Loss: 0.21135224401950836
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 4.956061840057373 | KNN Loss: 4.874513149261475 | CLS Loss: 0.08154855668544769
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 5.0474443435668945 | KNN Loss: 4.875492095947266 | CLS Loss: 0.17195230722427368
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 4.977890968322754 | KNN Loss: 4.8427324295043945 | CLS Loss: 0.13515834510326385
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 5.003618240356445 | KNN Loss: 4.8549323081970215 | CLS Loss: 0.14868596196174622
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 4.912004470825195 | KNN Loss: 4.81768798828125 | CLS Loss: 0.09431671351194382
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 4.937765598297119 | KNN Loss: 4.827059268951416 | CLS Loss: 0.11070647090673447
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 4.955239772796631 | KNN Loss: 4.84469461

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 4.912887096405029 | KNN Loss: 4.825107097625732 | CLS Loss: 0.08777984231710434
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 4.956603527069092 | KNN Loss: 4.823631286621094 | CLS Loss: 0.13297216594219208
Epoch: 007, Loss: 4.8936, Train: 0.9796, Valid: 0.9783, Best: 0.9783
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 4.85853910446167 | KNN Loss: 4.749813556671143 | CLS Loss: 0.10872567445039749
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 4.895758152008057 | KNN Loss: 4.828134059906006 | CLS Loss: 0.06762416660785675
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 4.900043487548828 | KNN Loss: 4.7993669509887695 | CLS Loss: 0.10067657381296158
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 4.841984748840332 | KNN Loss: 4.771867752075195 | CLS Loss: 0.07011718302965164
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 4.883410453796387 | KNN Loss: 4.786993503570557 | CLS Loss: 0.0964171290397644
Epoch 8 / 200 | iteration 50

Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 4.8051252365112305 | KNN Loss: 4.732942581176758 | CLS Loss: 0.07218284159898758
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 4.853011608123779 | KNN Loss: 4.76618766784668 | CLS Loss: 0.0868237093091011
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 4.826859474182129 | KNN Loss: 4.7362589836120605 | CLS Loss: 0.09060072153806686
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 4.8144049644470215 | KNN Loss: 4.770278453826904 | CLS Loss: 0.04412638023495674
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 4.824174880981445 | KNN Loss: 4.756758213043213 | CLS Loss: 0.06741679459810257
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 4.837057590484619 | KNN Loss: 4.7804694175720215 | CLS Loss: 0.05658822879195213
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 4.781062126159668 | KNN Loss: 4.74584436416626 | CLS Loss: 0.03521760553121567
Epoch 11 / 200 | iteration 130 / 171 | Total Loss: 4.897092342376709 | KNN Loss: 4.798

Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 4.756664752960205 | KNN Loss: 4.704980850219727 | CLS Loss: 0.051684118807315826
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 4.789742946624756 | KNN Loss: 4.739541053771973 | CLS Loss: 0.05020182952284813
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 4.834897994995117 | KNN Loss: 4.767545700073242 | CLS Loss: 0.06735212355852127
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 4.801946640014648 | KNN Loss: 4.755081653594971 | CLS Loss: 0.046865154057741165
Epoch: 014, Loss: 4.8062, Train: 0.9858, Valid: 0.9823, Best: 0.9823
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 4.761838436126709 | KNN Loss: 4.723233699798584 | CLS Loss: 0.038604702800512314
Epoch 15 / 200 | iteration 10 / 171 | Total Loss: 4.817720890045166 | KNN Loss: 4.738092422485352 | CLS Loss: 0.07962839305400848
Epoch 15 / 200 | iteration 20 / 171 | Total Loss: 4.823766231536865 | KNN Loss: 4.777785301208496 | CLS Loss: 0.045981090515851974
Epoch 15 / 200

...

Epoch 89 / 200 | iteration 120 / 171 | Total Loss: 4.710534572601318 | KNN Loss: 4.706701755523682 | CLS Loss: 0.0038326869253069162
Epoch 89 / 200 | iteration 130 / 171 | Total Loss: 4.721573352813721 | KNN Loss: 4.687057971954346 | CLS Loss: 0.034515202045440674
Epoch 89 / 200 | iteration 140 / 171 | Total Loss: 4.664742946624756 | KNN Loss: 4.656628608703613 | CLS Loss: 0.008114155381917953
Epoch 89 / 200 | iteration 150 / 171 | Total Loss: 4.746030330657959 | KNN Loss: 4.723965167999268 | CLS Loss: 0.022065024822950363
Epoch 89 / 200 | iteration 160 / 171 | Total Loss: 4.761634826660156 | KNN Loss: 4.739597320556641 | CLS Loss: 0.022037314251065254
Epoch 89 / 200 | iteration 170 / 171 | Total Loss: 4.790550708770752 | KNN Loss: 4.7768659591674805 | CLS Loss: 0.01368456706404686
Epoch: 089, Loss: 4.7135, Train: 0.9946, Valid: 0.9858, Best: 0.9872
Epoch 90 / 200 | iteration 0 / 171 | Total Loss: 4.6970906257629395 | KNN Loss: 4.680536270141602 | CLS Loss: 0.01655414327979088
Epoch 90

Epoch 93 / 200 | iteration 10 / 171 | Total Loss: 4.691937446594238 | KNN Loss: 4.677088737487793 | CLS Loss: 0.01484874077141285
Epoch 93 / 200 | iteration 20 / 171 | Total Loss: 4.669137001037598 | KNN Loss: 4.64680290222168 | CLS Loss: 0.022334007546305656
Epoch 93 / 200 | iteration 30 / 171 | Total Loss: 4.840211868286133 | KNN Loss: 4.810022354125977 | CLS Loss: 0.03018931858241558
Epoch 93 / 200 | iteration 40 / 171 | Total Loss: 4.7588887214660645 | KNN Loss: 4.724839687347412 | CLS Loss: 0.03404882550239563
Epoch 93 / 200 | iteration 50 / 171 | Total Loss: 4.670864105224609 | KNN Loss: 4.652909278869629 | CLS Loss: 0.017954975366592407
Epoch 93 / 200 | iteration 60 / 171 | Total Loss: 4.672861099243164 | KNN Loss: 4.663883686065674 | CLS Loss: 0.008977356366813183
Epoch 93 / 200 | iteration 70 / 171 | Total Loss: 4.652175426483154 | KNN Loss: 4.64774751663208 | CLS Loss: 0.004427762236446142
Epoch 93 / 200 | iteration 80 / 171 | Total Loss: 4.735976696014404 | KNN Loss: 4.71878

Epoch 96 / 200 | iteration 80 / 171 | Total Loss: 4.7123565673828125 | KNN Loss: 4.669720649719238 | CLS Loss: 0.042636122554540634
Epoch 96 / 200 | iteration 90 / 171 | Total Loss: 4.744257926940918 | KNN Loss: 4.701762676239014 | CLS Loss: 0.042495302855968475
Epoch 96 / 200 | iteration 100 / 171 | Total Loss: 4.743129253387451 | KNN Loss: 4.708944797515869 | CLS Loss: 0.03418467566370964
Epoch 96 / 200 | iteration 110 / 171 | Total Loss: 4.729891300201416 | KNN Loss: 4.713491439819336 | CLS Loss: 0.016399793326854706
Epoch 96 / 200 | iteration 120 / 171 | Total Loss: 4.69455623626709 | KNN Loss: 4.666153430938721 | CLS Loss: 0.02840261161327362
Epoch 96 / 200 | iteration 130 / 171 | Total Loss: 4.66073751449585 | KNN Loss: 4.648351669311523 | CLS Loss: 0.012385695241391659
Epoch 96 / 200 | iteration 140 / 171 | Total Loss: 4.741384506225586 | KNN Loss: 4.713133335113525 | CLS Loss: 0.028251243755221367
Epoch 96 / 200 | iteration 150 / 171 | Total Loss: 4.713798522949219 | KNN Loss: 

Epoch 99 / 200 | iteration 150 / 171 | Total Loss: 4.685487270355225 | KNN Loss: 4.668654441833496 | CLS Loss: 0.016832629218697548
Epoch 99 / 200 | iteration 160 / 171 | Total Loss: 4.662221908569336 | KNN Loss: 4.657052040100098 | CLS Loss: 0.005169677548110485
Epoch 99 / 200 | iteration 170 / 171 | Total Loss: 4.832718849182129 | KNN Loss: 4.808174133300781 | CLS Loss: 0.02454492822289467
Epoch: 099, Loss: 4.7115, Train: 0.9949, Valid: 0.9858, Best: 0.9875
Epoch 100 / 200 | iteration 0 / 171 | Total Loss: 4.812679767608643 | KNN Loss: 4.8079423904418945 | CLS Loss: 0.004737257957458496
Epoch 100 / 200 | iteration 10 / 171 | Total Loss: 4.801422119140625 | KNN Loss: 4.781116008758545 | CLS Loss: 0.020306140184402466
Epoch 100 / 200 | iteration 20 / 171 | Total Loss: 4.685364246368408 | KNN Loss: 4.6790995597839355 | CLS Loss: 0.0062647550366818905
Epoch 100 / 200 | iteration 30 / 171 | Total Loss: 4.678440093994141 | KNN Loss: 4.655169486999512 | CLS Loss: 0.02327064424753189
Epoch 1

Epoch 103 / 200 | iteration 30 / 171 | Total Loss: 4.692645072937012 | KNN Loss: 4.679912567138672 | CLS Loss: 0.01273231953382492
Epoch 103 / 200 | iteration 40 / 171 | Total Loss: 4.82916259765625 | KNN Loss: 4.769058704376221 | CLS Loss: 0.06010374054312706
Epoch 103 / 200 | iteration 50 / 171 | Total Loss: 4.667762756347656 | KNN Loss: 4.663081169128418 | CLS Loss: 0.004681488964706659
Epoch 103 / 200 | iteration 60 / 171 | Total Loss: 4.683344841003418 | KNN Loss: 4.674220561981201 | CLS Loss: 0.00912412442266941
Epoch 103 / 200 | iteration 70 / 171 | Total Loss: 4.748209476470947 | KNN Loss: 4.734986305236816 | CLS Loss: 0.013223184272646904
Epoch 103 / 200 | iteration 80 / 171 | Total Loss: 4.668698310852051 | KNN Loss: 4.646714687347412 | CLS Loss: 0.02198338508605957
Epoch 103 / 200 | iteration 90 / 171 | Total Loss: 4.70866584777832 | KNN Loss: 4.679040431976318 | CLS Loss: 0.02962525002658367
Epoch 103 / 200 | iteration 100 / 171 | Total Loss: 4.72878360748291 | KNN Loss: 4.

In [ ]:
test(model, test_data_iter, device)

In [ ]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [ ]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

In [ ]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [ ]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

In [ ]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")